# Enhanced Candlestick Chart with Plotly

This notebook demonstrates how to create an enhanced candlestick chart using the improved Chart class, random OHLCV data, and technical indicators.

In [1]:
# Import necessary libraries
import pandas as pd
from data.random_data import RandomOHLCV
from frame.frame import Frame
from chart.chart import Chart

In [2]:
# Generate random data
data = RandomOHLCV(
    freq='5 min',
    head_max=0.3,
    tail_max=0.3,
    start='2023-01-01',
    open_val=100.00,
    periods=300,
    open_rng=(-0.4, 0.4),
    close_rng=(-0.4, 0.4),
    vol_rng=(-50, 60),
    volatility_rng=(0, 0.02),
    volatility_dur=3,
    volatility_freq=50
)

df = data.get_dataframe()

c:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\random_data.py:87: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  self.data = RandDataStore(pd.date_range(start=self.start, periods=self.periods, freq=self.freq))


In [3]:
from data.data_manager import DataManager
from strategies.ta import MA, MACD

dm = DataManager(df)
dm.add_ta( MA('close', 20))
dm.add_ta( MA('close', 50))
dm.add_ta( MACD('close', 12, 26, 9))
dm.data

,open,high,low,close,volume,MA_cl_20,MA_cl_50,MACD_cl_12_26_9_MACD,MACD_cl_12_26_9_Signal,MACD_cl_12_26_9_Histogram
date,,,,,,,,,,
2023-01-01 00:00:00,99.66,99.81,99.23,99.39,500,NaN,NaN,0.000000,0.000000,0.000000
2023-01-01 00:05:00,99.32,99.62,99.18,99.47,14501,NaN,NaN,0.006382,0.001276,0.005105
2023-01-01 00:10:00,99.66,99.95,99.47,99.83,500,NaN,NaN,0.040027,0.009026,0.031000
2023-01-01 00:15:00,99.63,99.80,99.22,99.25,500,NaN,NaN,0.019663,0.011154,0.008509
2023-01-01 00:20:00,98.88,98.96,98.54,98.56,19498,NaN,NaN,-0.051558,-0.001389,-0.050170
...,...,...,...,...,...,...,...,...,...,...
2023-01-02 00:35:00,105.81,105.82,105.41,105.65,500,105.0060,104.3546,0.279549,0.258320,0.021229
2023-01-02 00:40:00,105.63,105.72,105.13,105.36,18669,105.0480,104.4212,0.273036,0.261263,0.011773
2023-01-02 00:45:00,105.21,105.34,105.15,105.21,500,105.0730,104.4880,0.252856,0.259582,-0.006726


In [4]:
# Create and display the enhanced chart
chart = Chart( title='Enhanced Random Data Chart', rowHeights=[0.2, 0.1, 0.7], height=800, width=1200)
chart.add_candles_and_volume(df)

# Add trading hours (assuming market hours are 9:30 AM to 4:00 PM)
chart.add_trading_hours(df, [('09:30', '16:00')])
from strategies.ta import MA, MACD

chart.refesh(df)
chart.add_ta(MA('close', 9).run(df), {'color': 'blue', 'width': 1}, 'line')
chart.add_ta(MA('close', 21).run(df), {'color': 'red', 'width': 1}, 'line')
chart.add_ta(MACD('close', 12, 26, 9).run(df), [
    {'color': 'green', 'width': 1},
    {'color': 'red', 'width': 1},
    {'color': 'blue', 'width': 1}], 'macd')

chart.show(width=1400)

In [5]:
# MACD('close', 12, 26, 9).run(df)

In [10]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Tuple
from data.data_manager import DataManager
from chart.chart import Chart
from strategies.ta import Indicator, MA, MACD, HPLP

@dataclass
class Frame:
    data: pd.DataFrame
    symbol: str
    trading_hours: List[Tuple[str, str]]
    indicators: List[Tuple[Indicator, Dict[str, Any]]] = field(default_factory=list)

    def __post_init__(self):
        self.dm = DataManager(self.data)
        self.chart = Chart(title=self.symbol, rowHeights=[0.2, 0.1, 0.7], height=800, width=800)
        self.chart.add_candles_and_volume(self.dm.data)
        self.chart.add_trading_hours(self.dm.data, self.trading_hours)

    def add_ta(self,  indicator: Indicator, style: Dict[str, Any], row: int = 1):
        self.dm.add_ta(indicator)
        self.indicators.append((indicator, style, row))

    # def update_data(self, new_data: Any):
    #     self.dm.update_data(new_data)
    #     self._update_chart()

    def _update_chart(self):
        for indicator, style, row in self.indicators:
            chart_type = "line"
            if indicator.__class__.__name__ in ["MACD"]: chart_type = "macd"
            if indicator.__class__.__name__ in ["HPLP"]: chart_type = "points"
            self.chart.add_ta(self.dm.data[indicator.names], style, chart_type, row)


    def plot(self, width:int=None, height:int=None):
        self._update_chart()
        self.chart.show(width=width, height=height)

    def plot_refresh(self):
        self.chart.refesh()
        self._update_chart()
        self.chart.show()

f5min = Frame(df, '5min', [('09:30', '16:00')])
f5min.add_ta(MA('close', 9), {'color': 'blue', 'width': 1})
f5min.add_ta(MA('close', 21), {'color': 'red', 'width': 1})
f5min.add_ta(MACD('close', 12, 26, 9), [
    {'color': 'green', 'width': 1},
    {'color': 'red', 'width': 1},
    {'color': 'blue', 'width': 1}], row=3)
f5min.add_ta(HPLP(hi_col='high', lo_col='low', span=20), [
    {'color': 'green', 'size': 10, 'opacity': 0.6},
    {'color': 'red', 'size': 10, 'opacity': 0.6}], row=1)
f5min.plot(width=1400)

In [7]:
f5min.indicators[0].name

AttributeError: 'tuple' object has no attribute 'name'